In [ ]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse

# Preprocess the Image (Grayscale and Divisible by 4)
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    height, width = image.shape
    height, width = height - height % 4, width - width % 4  # Make dimensions divisible by 4
    return image[:height, :width]

# Compress a Block Using AMBTC
def compress_block(block):
    mean = np.mean(block)
    binary_map = (block >= mean).astype(int)
    high_value = np.mean(block[binary_map == 1]) if np.sum(binary_map) > 0 else mean
    low_value = np.mean(block[binary_map == 0]) if np.sum(binary_map) < block.size else mean
    return binary_map, high_value, low_value

# Embed Data Using Difference Expansion
def embedding(block, binary_data):
    p1, p2 = block.flatten()[:2]
    d = p2 - p1
    secret_bit = int(binary_data[0])
    expanded_d = 2 * d + secret_bit
    p2_new = p1 + expanded_d
    flat_block = block.flatten()
    flat_block[:2] = [p1, p2_new]
    return flat_block.reshape(block.shape)

# Extract Data Using Reverse Difference Expansion
def extraction(block):
    p1, p2 = block.flatten()[:2]
    d_expanded = p2 - p1
    secret_bit = d_expanded % 2
    original_d = d_expanded // 2
    p2_original = p1 + original_d
    flat_block = block.flatten()
    flat_block[:2] = [p1, p2_original]
    return secret_bit, flat_block.reshape(block.shape)

# Embed Binary Data into the Entire Image
def embed_image(image, binary_data):
    height, width = image.shape
    binary_data = iter(binary_data)
    for i in range(0, height, 4):
        for j in range(0, width, 4):
            block = image[i:i+4, j:j+4]
            try:
                bit = next(binary_data)
                modified_block = embedding(block, bit)
                image[i:i+4, j:j+4] = modified_block
            except StopIteration:
                break
    return image

# Extract Binary Data from the Entire Image
def extract_image(image):
    height, width = image.shape
    extracted_data = []
    for i in range(0, height, 4):
        for j in range(0, width, 4):
            block = image[i:i+4, j:j+4]
            bit, _ = extraction(block)
            extracted_data.append(str(bit))
    return "".join(extracted_data)

# Evaluate PSNR and MSE
def evaluate_image_quality(original, embedded):
    mse_value = mse(original, embedded)
    psnr_value = psnr(original, embedded)
    return mse_value, psnr_value

# Main Workflow
def main(image_path, binary_data):
    image = preprocess_image(image_path)
    embedded_image = embed_image(image.copy(), binary_data)
    extracted_data = extract_image(embedded_image)
    mse_value, psnr_value = evaluate_image_quality(image, embedded_image)
    return embedded_image, extracted_data, mse_value, psnr_value

# Example Usage
if __name__ == "__main__":
    image_path = "example_image.jpg"  # Replace with your image path
    binary_data = "1010101010101010" * 100  # Example binary data
    embedded_image, extracted_data, mse_value, psnr_value = main(image_path, binary_data)
    print(f"MSE: {mse_value}, PSNR: {psnr_value} dB")
    print(f"Extracted Data (First 100 Bits): {extracted_data[:100]}")
    cv2.imwrite("embedded_image.jpg", embedded_image)
